In [140]:
'''Data encode and decode testing'''

import numpy as np
import tensorflow as tf

import input_pipeline



[ 10  98  10  96  10   6 114 101  99 111 114 100  18  86  10  84  10  82
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]


82

In [2]:
'''Plotting'''
from matplotlib import pyplot as plot

ModuleNotFoundError: No module named 'matplotlib'